In [ ]:
!pip install PyPDF2 scikit-learn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.6/232.6 kB 3.4 MB/s eta 0:00:00


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
import re
import pickle
import nltk
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from joblib import dump
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

# **Load Data**

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/PA_Kelompok 3/Resume2.csv')
data.head()

,ID,Resume_str,Resume_html,Category
0,16852973,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,"<div class=""fontsize fontface vmargins hmargin...",HR
1,22323967,"HR SPECIALIST, US HR OPERATIONS ...","<div class=""fontsize fontface vmargins hmargin...",HR
2,33176873,HR DIRECTOR Summary Over 2...,"<div class=""fontsize fontface vmargins hmargin...",HR
3,27018550,HR SPECIALIST Summary Dedica...,"<div class=""fontsize fontface vmargins hmargin...",HR
4,17812897,HR MANAGER Skill Highlights ...,"<div class=""fontsize fontface vmargins hmargin...",HR


In [ ]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 677 entries, 0 to 676
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   ID           677 non-null    int64 
 1   Resume_str   677 non-null    object
 2   Resume_html  676 non-null    object
 3   Category     677 non-null    object
dtypes: int64(1), object(3)
memory usage: 21.3+ KB


In [ ]:
data.drop(['Resume_html'], axis = 1, inplace = True)
data.drop(['ID'], axis = 1, inplace = True)
data

,Resume_str,Category
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR
1,"HR SPECIALIST, US HR OPERATIONS ...",HR
2,HR DIRECTOR Summary Over 2...,HR
3,HR SPECIALIST Summary Dedica...,HR
4,HR MANAGER Skill Highlights ...,HR
...,...,...
672,GENERAL ACCOUNTANT Professional...,ACCOUNTANT
673,STAFF ACCOUNTANT Professional S...,ACCOUNTANT
674,GENERAL ACCOUNTANT Summary ...,ACCOUNTANT
675,PROJECT ACCOUNTANT Career F...,ACCOUNTANT


In [ ]:
label_counts = data['Category'].value_counts()
label = data['Category'].nunique()
label_counts

BUSINESS-DEVELOPMENT    119
ENGINEERING             118
ACCOUNTANT              118
SALES                   116
HR                      110
DIGITAL-MEDIA            96
Name: Category, dtype: int64

In [ ]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()

data['Category_encoded'] = label_encoder.fit_transform(data['Category'])

# Menampilkan dataset yang telah diencode
print(data[['Category', 'Category_encoded']])

       Category  Category_encoded
0            HR                 4
1            HR                 4
2            HR                 4
3            HR                 4
4            HR                 4
..          ...               ...
672  ACCOUNTANT                 0
673  ACCOUNTANT                 0
674  ACCOUNTANT                 0
675  ACCOUNTANT                 0
676  ACCOUNTANT                 0

[677 rows x 2 columns]


In [ ]:
data

,Resume_str,Category,Category_encoded
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR,4
1,"HR SPECIALIST, US HR OPERATIONS ...",HR,4
2,HR DIRECTOR Summary Over 2...,HR,4
3,HR SPECIALIST Summary Dedica...,HR,4
4,HR MANAGER Skill Highlights ...,HR,4
...,...,...,...
672,GENERAL ACCOUNTANT Professional...,ACCOUNTANT,0
673,STAFF ACCOUNTANT Professional S...,ACCOUNTANT,0
674,GENERAL ACCOUNTANT Summary ...,ACCOUNTANT,0
675,PROJECT ACCOUNTANT Career F...,ACCOUNTANT,0


# **Text Preprocessing**

**Case Folding & Penghapusan Tanda Baca**

In [ ]:
import re

def casefolding(text):
  text = text.lower()
  text = re.sub(r'https?://\S+|www\.\S+', '', text)
  text = re.sub(r'[-+]?[0-9]+', '', text)
  text = re.sub(r'[^\w\s]','', text)
  text = text.strip()
  return text

In [ ]:
raw_sample = data['Resume_str'].iloc[5]
case_folding = casefolding(raw_sample)

print('Raw data\t: ', raw_sample)
print('Case folding\t: ', case_folding)

Raw data	:           HR GENERALIST       Summary     Dedicated and focused Administrative Assistant who excels at prioritizing, completing multiple tasks simultaneously and following through to achieve project goals. Seeking a role of increased responsibility and authority.       Highlights         Microsoft Office proficiency  Excel spreadsheets  Meticulous attention to detail  Results-oriented  Self-directed      Time management  Professional and mature  Self-starter  Legal administrative support            Experience     11/2008   to   08/2014     HR Generalist    Company Name   －   City  ,   State      Managed visa related employment processes for all non-immigrant faculty and staff.  Improved productivity and enhanced visa related services.  Improved operational structure by developing guidelines and tools for internal and external administration of non-immigrant employment procedures  Reduced internal employment authorization processing times by approximately 30 percent.  Prepare

**Stop Word**

In [ ]:
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.corpus import stopwords

stopwords_eng = stopwords.words('english')

In [ ]:
len(stopwords_eng)

179

In [ ]:
# Lihat daftar stopword yang disediakan NLTK
stopwords_eng[:179]

['i',
 'me',
 'my',
 'myself',
 'we',
 'our',
 'ours',
 'ourselves',
 'you',
 "you're",
 "you've",
 "you'll",
 "you'd",
 'your',
 'yours',
 'yourself',
 'yourselves',
 'he',
 'him',
 'his',
 'himself',
 'she',
 "she's",
 'her',
 'hers',
 'herself',
 'it',
 "it's",
 'its',
 'itself',
 'they',
 'them',
 'their',
 'theirs',
 'themselves',
 'what',
 'which',
 'who',
 'whom',
 'this',
 'that',
 "that'll",
 'these',
 'those',
 'am',
 'is',
 'are',
 'was',
 'were',
 'be',
 'been',
 'being',
 'have',
 'has',
 'had',
 'having',
 'do',
 'does',
 'did',
 'doing',
 'a',
 'an',
 'the',
 'and',
 'but',
 'if',
 'or',
 'because',
 'as',
 'until',
 'while',
 'of',
 'at',
 'by',
 'for',
 'with',
 'about',
 'against',
 'between',
 'into',
 'through',
 'during',
 'before',
 'after',
 'above',
 'below',
 'to',
 'from',
 'up',
 'down',
 'in',
 'out',
 'on',
 'off',
 'over',
 'under',
 'again',
 'further',
 'then',
 'once',
 'here',
 'there',
 'when',
 'where',
 'why',
 'how',
 'all',
 'any',
 'both',
 'each

In [ ]:

more_stopword = ['summary']                    # Tambahkan kata lain dalam daftar stopword
stopwords_eng = stopwords_eng + more_stopword

def remove_stop_words(text):
  clean_words = []
  text = text.split()
  for word in text:
      if word not in stopwords_eng:
          clean_words.append(word)
  return " ".join(clean_words)

In [ ]:
raw_sample = data['Resume_str'].iloc[5]
case_folding = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)

print('Raw data\t\t: ', raw_sample)
print('Case folding\t\t: ', case_folding)
print('Stopword removal\t: ', stopword_removal)

Raw data		:           HR GENERALIST       Summary     Dedicated and focused Administrative Assistant who excels at prioritizing, completing multiple tasks simultaneously and following through to achieve project goals. Seeking a role of increased responsibility and authority.       Highlights         Microsoft Office proficiency  Excel spreadsheets  Meticulous attention to detail  Results-oriented  Self-directed      Time management  Professional and mature  Self-starter  Legal administrative support            Experience     11/2008   to   08/2014     HR Generalist    Company Name   －   City  ,   State      Managed visa related employment processes for all non-immigrant faculty and staff.  Improved productivity and enhanced visa related services.  Improved operational structure by developing guidelines and tools for internal and external administration of non-immigrant employment procedures  Reduced internal employment authorization processing times by approximately 30 percent.  Prepar

**Lemmatization**

In [ ]:
from nltk.stem import WordNetLemmatizer

nltk.download('punkt')
nltk.download('wordnet')

lemmatizer = WordNetLemmatizer()

def lemmatization(text):
  text = lemmatizer.lemmatize(text)
  return text

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


In [ ]:
raw_sample = data['Resume_str'].iloc[5]
case_folding = casefolding(raw_sample)
stopword_removal = remove_stop_words(case_folding)
text_lemmatization = lemmatization(stopword_removal)

print('Raw data\t\t: ', raw_sample)
print('Case folding\t\t: ', case_folding)
print('Stopword removal\t: ', stopword_removal)
print('Lemmatization\t\t: ', text_lemmatization)

Raw data		:           HR GENERALIST       Summary     Dedicated and focused Administrative Assistant who excels at prioritizing, completing multiple tasks simultaneously and following through to achieve project goals. Seeking a role of increased responsibility and authority.       Highlights         Microsoft Office proficiency  Excel spreadsheets  Meticulous attention to detail  Results-oriented  Self-directed      Time management  Professional and mature  Self-starter  Legal administrative support            Experience     11/2008   to   08/2014     HR Generalist    Company Name   －   City  ,   State      Managed visa related employment processes for all non-immigrant faculty and staff.  Improved productivity and enhanced visa related services.  Improved operational structure by developing guidelines and tools for internal and external administration of non-immigrant employment procedures  Reduced internal employment authorization processing times by approximately 30 percent.  Prepar

**Text Preprocessing Pipeline**

In [ ]:
# Buat fungsi untuk menggabungkan seluruh langkah text preprocessing
def text_preprocessing_process(text):
  text = casefolding(text)
  text = remove_stop_words(text)
  text = lemmatization(text)
  return text

In [ ]:
%%time
data['clean_teks'] = data['Resume_str'].apply(text_preprocessing_process)

CPU times: user 1.53 s, sys: 0 ns, total: 1.53 s
Wall time: 1.54 s


In [ ]:
data

,Resume_str,Category,Category_encoded,clean_teks
0,HR ADMINISTRATOR/MARKETING ASSOCIATE\...,HR,4,hr administratormarketing associate hr adminis...
1,"HR SPECIALIST, US HR OPERATIONS ...",HR,4,hr specialist us hr operations versatile media...
2,HR DIRECTOR Summary Over 2...,HR,4,hr director years experience recruiting plus y...
3,HR SPECIALIST Summary Dedica...,HR,4,hr specialist dedicated driven dynamic years c...
4,HR MANAGER Skill Highlights ...,HR,4,hr manager skill highlights hr skills hr depar...
...,...,...,...,...
672,GENERAL ACCOUNTANT Professional...,ACCOUNTANT,0,general accountant professional around years e...
673,STAFF ACCOUNTANT Professional S...,ACCOUNTANT,0,staff accountant professional currently gradua...
674,GENERAL ACCOUNTANT Summary ...,ACCOUNTANT,0,general accountant years experienced professio...
675,PROJECT ACCOUNTANT Career F...,ACCOUNTANT,0,project accountant career focus dedicated focu...


**Save Preprocessing**

In [ ]:
# Simpan data yang telah melalui text preprocessing agar kita tidak perlu menjalankan proses tersebut mulai awal (Opsional)
data.to_csv('/content/drive/MyDrive/PA_Kelompok 3/clean_data.csv')

# **Feature Engineering**

In [ ]:
X = data['clean_teks']
y = data['Category_encoded']

In [ ]:
X

0      hr administratormarketing associate hr adminis...
1      hr specialist us hr operations versatile media...
2      hr director years experience recruiting plus y...
3      hr specialist dedicated driven dynamic years c...
4      hr manager skill highlights hr skills hr depar...
                             ...                        
672    general accountant professional around years e...
673    staff accountant professional currently gradua...
674    general accountant years experienced professio...
675    project accountant career focus dedicated focu...
676    senior accountant highly competent motivated r...
Name: clean_teks, Length: 677, dtype: object

In [ ]:
y

0      4
1      4
2      4
3      4
4      4
      ..
672    0
673    0
674    0
675    0
676    0
Name: Category_encoded, Length: 677, dtype: int64

**Feature Extraction (TF-IDF)**

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf_idf = TfidfVectorizer(ngram_range=(1,1))
tf_idf.fit(X)

TfidfVectorizer()

In [ ]:
# Melihat Jumlah Fitur
print(len(tf_idf.get_feature_names_out()))

21920


In [ ]:
# Melihat fitur-fitur apa saja yang ada di dalam corpus
print(tf_idf.get_feature_names_out())

['_______________________________________________________________________'
 '______________________________________________________________________________'
 '_______________________________________________________________________________'
 ... 'µpg' 'étiquette' 'òwaterfalló']


In [ ]:
# Melihat matriks jumlah token

X_tf_idf = tf_idf.transform(X).toarray()
X_tf_idf

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]])

In [ ]:
# Melihat matriks jumlah token menggunakan TF IDF

data_tf_idf = pd.DataFrame(X_tf_idf, columns=tf_idf.get_feature_names_out())
data_tf_idf

,_______________________________________________________________________,______________________________________________________________________________,_______________________________________________________________________________,aa,aaa,aac,aaeeo,aalto,aam,aap,...,zoom,zoos,zpro,ztv,zuckerman,zumper,zypher,µpg,étiquette,òwaterfalló
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
673,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
674,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
675,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
with open('tf_idf_feature.pickle', 'wb') as output:
  pickle.dump(X_tf_idf, output)

In [ ]:
# Mengubah nilai data tabular tf-idf menjadi array agar dapat dijalankan pada proses seleksi fitur
X = np.array(data_tf_idf)
y = np.array(y)

**Save Tf-Idf**

In [ ]:
pickle.dump(X_tf_idf, open('/content/drive/MyDrive/PA_Kelompok 3/tf_idf_feature.pickle', 'wb'))

**Feature Selection (Chi Square)**

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

# Ten features with highest chi-squared statistics are selected
chi2_features = SelectKBest(chi2, k=1000)
X_kbest_features = chi2_features.fit_transform(X, y)

# Reduced features
print('Original feature number:', X.shape[1])
print('Reduced feature number:', X_kbest_features.shape[1])

Original feature number: 21920
Reduced feature number: 1000


In [ ]:
# chi2_features.scores_ adalah nilai chi-square, semakin tinggi nilainya maka semakin baik fiturnya
data_chi2 = pd.DataFrame(chi2_features.scores_, columns=['nilai'])
data_chi2

,nilai
0,0.297120
1,0.139683
2,0.139683
3,0.307804
4,0.152878
...,...
21915,0.282288
21916,0.217285
21917,0.162149
21918,0.329939


In [ ]:
# Menampilkan fitur beserta nilainya
feature = tf_idf.get_feature_names_out()
data_chi2['fitur'] = feature
data_chi2

,nilai,fitur
0,0.297120,______________________________________________...
1,0.139683,______________________________________________...
2,0.139683,______________________________________________...
3,0.307804,aa
4,0.152878,aaa
...,...,...
21915,0.282288,zumper
21916,0.217285,zypher
21917,0.162149,µpg
21918,0.329939,étiquette


In [ ]:
# Mengurutkan fitur terbaik
data_chi2.sort_values(by='nilai', ascending=False)

,nilai,fitur
9168,94.847852,hr
150,84.943796,accounting
6597,69.416780,engineering
138,55.459967,accountant
7505,52.219060,financial
...,...,...
12685,0.023283,multi
15893,0.020463,ratings
729,0.017825,allow
38,0.017676,ability


In [ ]:
# Menampilkan mask pada feature yang diseleksi
# False berarti fitur tidak terpilih dan True berarti fitur terpilih
mask = chi2_features.get_support()
mask

array([False, False, False, ..., False, False, False])

In [ ]:
# Menampilkan fitur-fitur terpilih berdasarkan mask atau nilai tertinggi yang sudah dikalkulasi pada Chi-Square
new_feature = []

for bool, f in zip(mask, feature):
  if bool:
    new_feature.append(f)
  selected_feature = new_feature

selected_feature

['ab',
 'abbott',
 'absence',
 'absences',
 'account',
 'accountancy',
 'accountant',
 'accountants',
 'accounting',
 'accounts',
 'accrual',
 'accruals',
 'accrued',
 'accuracy',
 'accurate',
 'accurately',
 'ach',
 'achieved',
 'acoustic',
 'across',
 'action',
 'actions',
 'actual',
 'ad',
 'ada',
 'addressed',
 'adjustments',
 'administer',
 'administered',
 'administering',
 'administration',
 'administrative',
 'adobe',
 'adp',
 'ads',
 'advertising',
 'adwords',
 'affiliate',
 'affirmative',
 'agency',
 'agile',
 'aging',
 'air',
 'aircraft',
 'alerted',
 'allen',
 'allocations',
 'always',
 'alza',
 'analysis',
 'analytics',
 'analyzer',
 'analyzers',
 'angies',
 'animals',
 'animation',
 'annual',
 'answer',
 'answered',
 'ansys',
 'ap',
 'api',
 'appealing',
 'applicant',
 'applicants',
 'appraisal',
 'ar',
 'architected',
 'architecture',
 'art',
 'articles',
 'ascertain',
 'asking',
 'aspnet',
 'assembly',
 'asset',
 'assets',
 'assimilation',
 'assistant',
 'associate',
 '

In [ ]:
# Cara melihat vocab yang dihasilkan oleh TF_IDF

kbest_feature = {} # Buat dictionary kosong

for (k,v) in tf_idf.vocabulary_.items():    # Iterasi untuk mengulangi vocab yang dihasilkan TF_IDF
  if k in selected_feature:                 # Cek apakah fitur termasuk k fitur yang diseleksi
    kbest_feature[k] = v                    # Jika iya, simpan fitur tersebut pada dictionary kosong diatas

In [ ]:
kbest_feature

{'hr': 9168,
 'associate': 1371,
 'customer': 4821,
 'service': 17682,
 'manager': 11576,
 'management': 11555,
 'customerfocused': 4828,
 'satisfaction': 17271,
 'marketing': 11725,
 'conflict': 4070,
 'resolution': 16609,
 'training': 20049,
 'development': 5418,
 'client': 3442,
 'relations': 16310,
 'general': 8180,
 'server': 17674,
 'systems': 19251,
 'system': 19240,
 'sales': 17170,
 'strategies': 18788,
 'inventory': 10084,
 'control': 4256,
 'safety': 17137,
 'leadership': 10887,
 'performance': 14327,
 'policies': 14767,
 'employment': 6493,
 'compensation': 3843,
 'labor': 10714,
 'benefits': 1955,
 'employee': 6474,
 'prepares': 15081,
 'notices': 13255,
 'related': 16305,
 'documentation': 5855,
 'records': 16074,
 'insurance': 9895,
 'personnel': 14386,
 'transactions': 20072,
 'hires': 8998,
 'promotions': 15437,
 'transfers': 20091,
 'terminations': 19574,
 'reporting': 16490,
 'issues': 10235,
 'programs': 15388,
 'leave': 10935,
 'absence': 59,
 'collateral': 3650,
 

In [ ]:
# Menampilkan fitur-fitur yang sudah diseleksi
# Hanya k fitur yang terpilih sesuai parameter k yang ditentukan sebelumnya

data_selected_feature = pd.DataFrame(X_kbest_features, columns=selected_feature)
data_selected_feature

,ab,abbott,absence,absences,account,accountancy,accountant,accountants,accounting,accounts,...,workforce,workplace,writers,wrote,ws,wu,year,yearend,yoga,youtube
0,0.0,0.0,0.043117,0.0,0.000000,0.0,0.000000,0.000000,0.046180,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
1,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
2,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
3,0.0,0.0,0.067630,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
4,0.0,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,...,0.022869,0.0,0.0,0.020752,0.000000,0.0,0.013688,0.000000,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
672,0.0,0.0,0.000000,0.0,0.090199,0.0,0.184437,0.000000,0.179778,0.126945,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0
673,0.0,0.0,0.000000,0.0,0.085389,0.0,0.103467,0.000000,0.129670,0.044509,...,0.000000,0.0,0.0,0.000000,0.025427,0.0,0.031478,0.034279,0.0,0.0
674,0.0,0.0,0.000000,0.0,0.031122,0.0,0.106063,0.000000,0.159508,0.131403,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.068839,0.056223,0.0,0.0
675,0.0,0.0,0.000000,0.0,0.000000,0.0,0.064740,0.000000,0.072120,0.000000,...,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.017508,0.000000,0.0,0.0


In [ ]:
with open('kbest_feature.pickle', 'wb') as output:
  pickle.dump(kbest_feature, output)

**Save kbest_feature**

In [ ]:
pickle.dump(kbest_feature, open('/content/drive/MyDrive/PA_Kelompok 3/kbest_feature.pickle', 'wb'))

# **Modelling (Machine Learning)**

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from joblib import dump

In [ ]:
# Proses memisahkan data uji dan data latih. Perbandingan 80% untuk data latih, 20% untuk data uji

X_train, X_test, y_train, y_test = train_test_split(X_kbest_features, y, test_size=0.2, random_state=40)

In [ ]:
algorithm = MultinomialNB()
model = algorithm.fit(X_train, y_train)

In [ ]:
model_pred = model.predict(X_test)

In [ ]:
# Tampilkan hasil prediksi label dari model
model_pred

array([0, 1, 1, 5, 4, 5, 4, 3, 0, 3, 5, 5, 1, 1, 0, 1, 2, 3, 2, 5, 5, 0,
       0, 3, 0, 0, 2, 0, 5, 0, 3, 4, 0, 2, 3, 5, 4, 1, 1, 3, 3, 0, 1, 0,
       3, 5, 3, 5, 0, 3, 4, 3, 0, 5, 5, 5, 3, 1, 5, 5, 5, 5, 0, 4, 5, 0,
       5, 1, 4, 1, 4, 3, 5, 3, 1, 1, 4, 2, 4, 4, 1, 3, 1, 4, 4, 0, 3, 0,
       4, 0, 0, 5, 2, 1, 1, 3, 1, 5, 1, 4, 0, 3, 1, 2, 3, 1, 3, 2, 0, 1,
       1, 5, 2, 0, 1, 1, 1, 0, 4, 0, 3, 0, 1, 3, 2, 4, 0, 0, 4, 0, 4, 1,
       3, 0, 1, 2])

In [ ]:
# Tampilkan label sebenarnya pada data uji
y_test

array([0, 1, 1, 5, 4, 5, 4, 3, 0, 3, 2, 5, 1, 1, 0, 1, 2, 3, 2, 1, 5, 0,
       0, 2, 0, 0, 2, 0, 1, 2, 3, 4, 0, 2, 3, 5, 1, 1, 1, 3, 2, 0, 5, 0,
       3, 2, 3, 5, 0, 3, 4, 3, 1, 5, 5, 5, 3, 2, 5, 1, 5, 5, 0, 4, 5, 0,
       5, 5, 4, 1, 4, 5, 1, 3, 1, 1, 4, 2, 4, 4, 1, 3, 2, 4, 4, 0, 3, 0,
       4, 0, 0, 4, 2, 1, 1, 3, 5, 5, 1, 4, 0, 5, 3, 2, 3, 1, 3, 2, 4, 1,
       5, 5, 2, 0, 1, 1, 1, 0, 4, 0, 3, 0, 1, 3, 2, 4, 0, 0, 4, 0, 1, 1,
       3, 0, 1, 2])

In [ ]:
# Hitung jumlah data yang berhasil di prediksi model & jumlah data yang salah di prediksi
prediksi_benar = (model_pred == y_test).sum()
prediksi_salah = (model_pred != y_test).sum()

print('Jumlah prediksi benar\t:', prediksi_benar)
print('Jumlah prediksi salah\t:', prediksi_salah)

accuracy = prediksi_benar / (prediksi_benar + prediksi_salah)*100
print('Akurasi pengujian\t:', accuracy, '%')

Jumlah prediksi benar	: 113
Jumlah prediksi salah	: 23
Akurasi pengujian	: 83.08823529411765 %


In [ ]:
from sklearn.metrics import confusion_matrix

cm = confusion_matrix(y_test, model_pred)
print('Confusion matrix:\n', cm)

Confusion matrix:
 [[27  0  0  0  0  0]
 [ 1 22  0  0  2  4]
 [ 1  2 11  2  0  2]
 [ 0  1  0 20  0  0]
 [ 1  0  0  0 17  1]
 [ 0  4  0  2  0 16]]


In [ ]:
from sklearn.metrics import classification_report

print('Classification report:\n', classification_report(y_test, model_pred))

Classification report:
               precision    recall  f1-score   support

           0       0.90      1.00      0.95        27
           1       0.76      0.76      0.76        29
           2       1.00      0.61      0.76        18
           3       0.83      0.95      0.89        21
           4       0.89      0.89      0.89        19
           5       0.70      0.73      0.71        22

    accuracy                           0.83       136
   macro avg       0.85      0.82      0.83       136
weighted avg       0.84      0.83      0.83       136



In [ ]:
# Simpan model hasil traning
dump(model, filename='/content/drive/MyDrive/PA_Kelompok 3/model1.joblib')

['/content/drive/MyDrive/PA_Kelompok 3/model1.joblib']

**Simple Development**

In [ ]:
from joblib import load

# load model
model = load('/content/drive/MyDrive/PA_Kelompok 3/model1.joblib')

# load vocabulary dari TF_idf
vocab = pickle.load(open('kbest_feature.pickle', 'rb'))

In [ ]:
!pip install pymupdf

In [ ]:
import PyPDF2

def read_pdf(file_path):
    with open(file_path, 'rb') as file:
        pdf_reader = PyPDF2.PdfReader(file)
        num_pages = len(pdf_reader.pages)

        text = ''
        for page_num in range(num_pages):
            page = pdf_reader.pages[page_num]
            text += page.extract_text()

    return text

# upload file path .pdf
file_path = '/content/10399912.pdf'
pdf_text = read_pdf(file_path)

#klasifikasi
pre_input_text = text_preprocessing_process(pdf_text)   # lakukan text pre processing pada text input

tf_idf_vec = TfidfVectorizer(vocabulary=set(vocab))       # definisikan TF_IDF

result = model.predict(tf_idf_vec.fit_transform([pre_input_text]))  # Lakukan prediksi

if (result==0):
  divisi = 'Account'
elif (result==1):
  divisi = 'Business-Development'
elif (result==2):
  divisi = 'Digital-Media'
elif (result==3):
  divisi = 'Engineering'
elif (result==4):
  divisi = 'Human Resource'
elif (result==5):
  divisi = 'Sales'
else:
  divisi = 'Tidak Tersedia'

print('Hasil prediksi adalah', divisi)

Hasil prediksi adalah Human Resource


In [ ]:
#cetak CV
print(pdf_text)

HR PERSONNEL ASSISTANT
Summary
I am a U.S. citizen who is authorized to work in the US for any employer. I have worked 8 years as an Office Clerk, 2 years as a Student
Intern/Office Assistant, and 4 years as a Contractor. I am applying for the Data Entry Clerk position (Advert ID# 224278 Advert ID# 224278).
My skills and experiences include: Administrative Support, Auditing, File Management, Meeting Facilitation, Office Materials Management, &
Inventory Management.
Highlights
COMPUTER SKILLS: Microsoft Word, MS Excel, MS Outlook, MS PowerPoint, PeopleSoft. TYPING SKILLS: 40-60 WPM.
ADDITIONAL SKILLS: Administrative Support, Auditing, Clerical, Copy, Customer Service, Data Entry, Delivery, Documentation, Fax, File
Management, Letters, Meeting Facilitation, Organizational Skills, Proofreading, Receptionist, Research, Scanning, Scheduling, Secretarial,
Telephone Skills, Office Equipment Maintenance, and Inventory Management.
Experience
Company Name
 
City
 
, 
State
 
HR Personnel Assista